In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

from sklearn import linear_model, metrics, svm, neighbors, gaussian_process, ensemble, neural_network
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

from PIL import Image
from tqdm import tqdm
#from pandas_profiling import ProfileReport

import keras, re, string
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical

from nltk.sentiment.vader import SentimentIntensityAnalyzer

Using TensorFlow backend.


In [2]:
df = pd.read_csv("dataWithImagePaths.csv")

In [3]:
df.head()

,Unnamed: 0,totPurchaseAmt,street1,zip,property,ZillowURL,zpid,streetAddress,zipcode,city,state,latitude,longitude,dateSold,bathrooms,bedrooms,livingArea,yearBuilt,lotSize,homeType,homeStatus,photoCount,imageLink,daysOnZillow,isFeatured,shouldHighlight,brokerId,zestimate,rentZestimate,listing_sub_type,priceReduction,isUnmappable,rentalPetsFlags,mediumImageLink,isPreforeclosureAuction,homeStatusForHDP,festimate,isListingOwnedByCurrentSignedInAgent,isListingClaimedByCurrentSignedInUser,hiResImageLink,watchImageLink,tvImageLink,tvCollectionImageLink,tvHighResImageLink,zillowHasRightsToImages,desktopWebHdpImageLink,isNonOwnerOccupied,hideZestimate,isPremierBuilder,isZillowOwned,currency,country,taxAssessedValue,streetAddressOnly,unit,averageSchoolRating,description,price,priceForHDP,hoaFee,contactPhone,status,imgFilepath
0,0,152000.0,538 78th Avenue NE,55432.0,538 78th Avenue NE 55432,https://www.zillow.com/homedetails/538-78th-Av...,1475959,538 78th Ave NE,55432,Spring Lake Park,MN,45.108779,-93.257808,1569308400000,1.0,2.0,1300.0,1984,1225,CONDO,RECENTLY_SOLD,4,https://photos.zillowstatic.com/p_g/IS1fwt0hhf...,174,False,False,0,158110.0,1150.0,NaN,NaN,False,64.0,https://photos.zillowstatic.com/p_c/IS1fwt0hhf...,False,RECENTLY_SOLD,158110.0,False,False,https://photos.zillowstatic.com/p_f/IS1fwt0hhf...,https://photos.zillowstatic.com/p_j/IS1fwt0hhf...,https://photos.zillowstatic.com/p_m/IS1fwt0hhf...,https://photos.zillowstatic.com/p_l/IS1fwt0hhf...,https://photos.zillowstatic.com/p_n/IS1fwt0hhf...,True,https://photos.zillowstatic.com/p_h/IS1fwt0hhf...,NaN,False,False,False,USD,USA,-1.0,538 78th Ave NE,,4.000000,"Just move in! End unit, deck, double garage, w...",152000.0,152000.0,230.0,NaN,Completed,images/img_0.jpg
1,1,198000.0,14649 Iodine Ct NW,55303.0,14649 Iodine Ct NW 55303,https://www.zillow.com/homedetails/14649-Iodin...,1539563,14649 Iodine Ct NW,55303,Ramsey,MN,45.237189,-93.409535,1568271600000,2.0,4.0,1716.0,1996,10716,SINGLE_FAMILY,RECENTLY_SOLD,0,https://www.zillowstatic.com/static/images/nop...,215,False,False,0,285985.0,1775.0,NaN,NaN,False,192.0,https://www.zillowstatic.com/static/images/nop...,False,RECENTLY_SOLD,285985.0,False,False,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,True,https://www.zillowstatic.com/static/images/nop...,NaN,False,False,False,USD,USA,-1.0,14649 Iodine Ct NW,,5.333333,"14649 Iodine Ct NW, Ramsey, MN is a single fam...",198000.0,198000.0,NaN,NaN,Completed,no photo
2,2,130000.0,10921 Dogwood Street,55448.0,10921 Dogwood Street 55448,https://www.zillow.com/homedetails/10921-Dogwo...,1503295,10921 Dogwood St NW,55448,Coon Rapids,MN,45.168138,-93.271746,1566889200000,2.0,4.0,864.0,1963,12100,SINGLE_FAMILY,RECENTLY_SOLD,0,https://www.zillowstatic.com/static/images/nop...,179,False,False,0,132773.0,1675.0,NaN,NaN,False,64.0,https://www.zillowstatic.com/static/images/nop...,False,RECENTLY_SOLD,132773.0,False,False,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,https://www.zillowstatic.com/static/images/nop...,True,https://www.zillowstatic.com/static/images/nop...,False,False,False,False,USD,USA,-1.0,10921 Dogwood St NW,,5.666667,"10921 Dogwood St NW, Coon Rapids, MN is a sing...",130000.0,130000.0,NaN,NaN,Completed,no photo
3,3,415000.0,16931 Helium Street NW,55303.0,16931 Helium Street NW 55303,https://www.zillow.com/homedetails/16931-Heliu...,1500884,16931 Helium St NW,55303,Ramsey,MN,45.278217,-93.407533,1571209200000,2.0,3.0,3108.0,1985,120225,SINGLE_FAMILY,RECENTLY_SOLD,30,https://photos.zillowstatic.com/p_g/IS72penoji...,181,False,False,0,431995.0,2349.0,NaN,NaN,False,192.0,https://photos.zillowstatic.com/p_c/IS72penoji...,False,RECENTL

In [4]:
def get_size_statistics(data):
    heights = []
    widths = []
    img_count = 0
    for i, row in data.iterrows():
        if row["imgFilepath"] != "no photo" or row["imgFilepath"] != np.nan:
            try:
                imgData = np.array(Image.open(row["imgFilepath"]))
                heights.append(imgData.shape[0])
                widths.append(imgData.shape[1])
                img_count += 1
            except:
                data.at[i, "imgFilepath"] = "no photo"
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))

get_size_statistics(df)

Average Height: 234.0
Max Height: 234
Min Height: 234


Average Width: 316.0
Max Width: 316
Min Width: 316


In [5]:
noPhotoIndx = df[df["imgFilepath"] == "no photo"].index 

df.drop(noPhotoIndx, inplace=True)

df = df[df["totPurchaseAmt"] >= 50000]
df = df[df["totPurchaseAmt"] <= 5000000]

df = df[df["yearBuilt"] <= 2020]
df = df[df["yearBuilt"] >= 1940]

df = df[df["zipcode"] <= 56763]
df = df[df["zipcode"] >= 55001]

df = df[df["livingArea"] <= 30000]
df = df[df["livingArea"] >= 1000]

df.drop_duplicates("mediumImageLink", keep = "first", inplace = True)

# df = pd.get_dummies(data=df, columns=["bathrooms", "bedrooms", "yearBuilt", "zipcode"])

In [6]:
zillowPred = df[["totPurchaseAmt", "zestimate"]]

zillowPred.dropna(inplace=True)

44800.798621756876

In [7]:
df = df[
    ["totPurchaseAmt", "bathrooms", "latitude", "longitude",
     "bedrooms", "yearBuilt", "livingArea",
     "averageSchoolRating", "zipcode", "description", "imgFilepath"]
]

df.dropna(inplace=True)

df.head()

,totPurchaseAmt,bathrooms,latitude,longitude,bedrooms,yearBuilt,livingArea,averageSchoolRating,zipcode,description,imgFilepath
0,152000.0,1.0,45.108779,-93.257808,2.0,1984,1300.0,4.000000,55432,"Just move in! End unit, deck, double garage, w...",images/img_0.jpg
3,415000.0,2.0,45.278217,-93.407533,3.0,1985,3108.0,6.333333,55303,Stunning walk to Rum River and Creek running t...,images/img_3.jpg
5,262000.0,2.0,45.164166,-93.297836,3.0,1985,2158.0,4.666667,55433,"This large rambler is located on a beautiful, ...",images/img_5.jpg
6,280000.0,2.0,45.283700,-93.332023,3.0,1976,1993.0,7.333333,55304,You won't want to miss out on this beautiful h...,images/img_6.jpg
7,237000.0,2.0,45.105931,-93.261656,3.0,1959,2058.0,3.666667,55432,"Great, Updated, Ready To Move In, Starter Home...",images/img_7.jpg


In [8]:
def remove_mypunct(corpus):

    corpus = map(lambda x: re.sub(r"(#|@|http)\S+", "", x), corpus) #get rid of hashtags 
    corpus = map(lambda x: re.sub(r"(\t|\n|\v|…|“|”)", "", x), corpus) #get rid of whitesape
    corpus = map(lambda x:  re.sub(r"""\w*\d\w*""", ' ', x.lower()), corpus) #get rid of numbers
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    corpus = map(lambda x: punc_re.sub(' ', x), corpus)
    corpus = map(lambda x: re.sub('[\W_]+', " ", x), corpus) #get rid of emojis
    
    return list(corpus)

sid = SentimentIntensityAnalyzer()
score_tuples = []
score = []

for i in df.index.values:
    clean_house = remove_mypunct([df.description[i]])
    ss = sid.polarity_scores(clean_house[0])
    df.at[i, 'sentiment'] = float(ss['compound'])

df = df.drop("description", axis=1)

In [17]:
IMG_SIZE = 224

from tqdm import notebook

def predictPriceWithImage(data, model):
    imageData = []
    for i, row in notebook.tqdm(data.iterrows(), total=len(data)):
        img = Image.open(row["imgFilepath"])
        img = img.resize((IMG_SIZE, IMG_SIZE))
        img = np.array(img)
        img = np.array([i for i in img]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
        pred = model.predict(img)
        data.at[i, "imgPred"] = pred

model = keras.models.load_model("ZillowCNN.h5")

df["imgPred"] = 0

predictPriceWithImage(df, model)

In [18]:
df.head()

,totPurchaseAmt,bathrooms,latitude,longitude,bedrooms,yearBuilt,livingArea,averageSchoolRating,zipcode,imgFilepath,sentiment,imgPred
0,152000.0,1.0,45.108779,-93.257808,2.0,1984,1300.0,4.000000,55432,images/img_0.jpg,0.0516,353241
3,415000.0,2.0,45.278217,-93.407533,3.0,1985,3108.0,6.333333,55303,images/img_3.jpg,0.2617,282406
5,262000.0,2.0,45.164166,-93.297836,3.0,1985,2158.0,4.666667,55433,images/img_5.jpg,0.9136,372622
6,280000.0,2.0,45.283700,-93.332023,3.0,1976,1993.0,7.333333,55304,images/img_6.jpg,0.9428,259128
7,237000.0,2.0,45.105931,-93.261656,3.0,1959,2058.0,3.666667,55432,images/img_7.jpg,0.9869,239807


In [19]:
df = df.drop("imgFilepath", axis=1)

grouped = df.groupby('zipcode')
df = grouped.filter(lambda x: x['zipcode'].count() > 50)

In [20]:
df.head()

,totPurchaseAmt,bathrooms,latitude,longitude,bedrooms,yearBuilt,livingArea,averageSchoolRating,zipcode,sentiment,imgPred
0,152000.0,1.0,45.108779,-93.257808,2.0,1984,1300.0,4.000000,55432,0.0516,353241
3,415000.0,2.0,45.278217,-93.407533,3.0,1985,3108.0,6.333333,55303,0.2617,282406
5,262000.0,2.0,45.164166,-93.297836,3.0,1985,2158.0,4.666667,55433,0.9136,372622
6,280000.0,2.0,45.283700,-93.332023,3.0,1976,1993.0,7.333333,55304,0.9428,259128
7,237000.0,2.0,45.105931,-93.261656,3.0,1959,2058.0,3.666667,55432,0.9869,239807


In [21]:
X = df.drop("totPurchaseAmt", axis=1)
y = df["totPurchaseAmt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)  # apply same transformation to test data

In [74]:
model = Sequential()

n_cols = X_train.shape[1]
model.add(Dense(16, activation='relu', input_shape=(n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=["mae"])

In [75]:
model.fit(X_train, y_train, validation_split=0.1, epochs=100, batch_size=50, verbose=1)

Train on 10073 samples, validate on 1120 samples
Epoch 1/100
10073/10073 [==============================] - 1s 121us/step - loss: 143272.2202 - mae: 143272.2344 - val_loss: 65694.1965 - val_mae: 65694.2031
Epoch 2/100
10073/10073 [==============================] - 1s 91us/step - loss: 61265.7204 - mae: 61265.7266 - val_loss: 61436.3133 - val_mae: 61436.3125
Epoch 3/100
10073/10073 [==============================] - 1s 91us/step - loss: 57042.7538 - mae: 57042.7500 - val_loss: 56241.7219 - val_mae: 56241.7266
Epoch 4/100
10073/10073 [==============================] - 1s 92us/step - loss: 54957.5234 - mae: 54957.5312 - val_loss: 55902.8533 - val_mae: 55902.8555
Epoch 5/100
10073/10073 [==============================] - 1s 92us/step - loss: 53313.9557 - mae: 53313.9492 - val_loss: 53690.8458 - val_mae: 53690.8516
Epoch 6/100
10073/10073 [==============================] - 1s 91us/step - loss: 52961.2267 - mae: 52961.2305 - val_loss: 52763.1479 - val_mae: 52763.1445
Epoch 7/100
10073/10073 

In [76]:
y_pred = model.predict(X_test)

ourMAE = metrics.mean_absolute_error(y_test, y_pred)

print('Mean Absolute Error:', ourMAE)  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 45812.655696454094
Mean Squared Error: 5406799456.556153
Root Mean Squared Error: 73530.94217100821


In [77]:
zillowMAE = metrics.mean_absolute_error(zillowPred["totPurchaseAmt"], zillowPred["zestimate"])

print("Zillow Mean Absolute Error:", zillowMAE)
print("Zillow Mean Squared Error:", metrics.mean_squared_error(zillowPred["totPurchaseAmt"], zillowPred["zestimate"]))
print("Zillow Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(zillowPred["totPurchaseAmt"], zillowPred["zestimate"])))

Zillow Mean Absolute Error: 44800.798621756876
Zillow Mean Squared Error: 8888093062.314123
Zillow Root Mean Squared Error: 94276.68355597858


In [78]:
print("Error off Zillow Predictions:", ourMAE-zillowMAE)

Error off Zillow Predictions: 1011.8570746972182
